In [7]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
mdir = Path(Path(os.path.abspath('')).parent)
sys.path.append(str(mdir))
from copy import deepcopy

In [12]:
mdir/'data'/'train.csv'

PosixPath('/Users/denis.semenov/Desktop/Ra1fHak_DS/data/train.csv')

In [13]:
 train = pd.read_csv(mdir/'data'/'train.csv')

/Users/denis.semenov/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
train['city'].unique()

array(['Пермь', 'Шатура', 'Ярославль', ..., 'Липовицы', 'Черемушки',
       'Берсеневка'], dtype=object)

In [20]:
test = pd.read_csv(mdir/'data'/'test.csv')

In [22]:
test.sample(10)

,city,floor,id,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,...,reform_mean_year_building_1000,reform_mean_year_building_500,region,lat,lng,total_square,street,date,realty_type,price_type
1184,Томск,2.0,COL_363479,2,71,134,187,0,1,3,...,1934.475113,1936.260274,Томская область,56.495895,84.948275,720.587683,S9675,2020-10-25,10,1
2282,Пермь,NaN,COL_420790,7,41,126,245,0,0,0,...,1970.661157,1970.857143,Пермский край,58.003113,56.239762,173.896961,S20753,2020-12-06,10,1
306,Краснодар,"3 этаж, мансарда (4 этаж)",COL_308206,1,10,41,78,1,20,41,...,1968.975207,1967.900000,Краснодарский край,45.048659,38.956181,233.635659,S22411,2020-09-20,100,1
1575,Орёл,NaN,COL_384499,1,37,68,128,0,1,1,...,1965.197368,1960.051282,Орловская область,52.964570,36.075497,90.314249,S12680,2020-11-15,100,1
2109,Калуга,1.0,COL_412125,0,3,4,8,0,0,0,...,2013.250000,2012.000000,Калужская область,54.483670,36.198320,135.734445,S2001,2020-11-29,100,1
665,Красноярск,NaN,COL_330474,2,27,56,88,0,0,0,...,1996.348837,2012.565217,Красноярский край,56.037018,92.883097,265.122267,S13518,2020-10-04,10,1
2694,Санкт-Петербург,1,COL_443604,22,365,771,1221,0,8,10,...,1885.845953,1882.487805,Санкт-Петербург,59.933301,30.363862,76.644625,S5891,2020-12-20,110,1
2489,Петрозаводск,2,COL_432319,6,29,48,75,0,0,0,...,1967.268456,1975.000000,Карелия,61.776500,34.365534,78.900411,S13508,2020-12-13,100,1
475,Иркутск,NaN,COL_318209,6,90,160,230,0,0,0,...,1946.118577,1940.027778,Иркутская область,52.283810,104.283349,183.853240,S12711,2020-09-27,100,1
1076,Санкт-Петербург,3,COL_357040,4,56,108,237,0,0,0,...,1895.068376,1904.357143,Санкт-Петербург,59.910203,30.287477,674.086566,S18312,2020-10-25,100,1
